Colab에서 런타임 > 런타임 유형 변경 > 하드웨어 가속기에서 TPU 선택

In [6]:
# tpu 자원 얻기: resolver
import tensorflow as tf
import os

try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', resolver.master())
except ValueError:
    print('Not connected to a TPU runtime. Using CPU/GPU strategy instead.')
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
    # If TPU is not available, this will default to CPU/GPU

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

Running on TPU  


In [7]:
strategy = tf.distribute.TPUStrategy(resolver)

In [8]:
# test model
def create_model():
    return tf.keras.Sequential(
        [tf.keras.layers.Conv2D(256, 4, activation='relu', input_shape=(28, 28, 1)),
         tf.keras.layers.Conv2D(256, 3, activation='relu'),
         tf.keras.layers.Flatten(),
         tf.keras.layers.Dense(256, activation='relu'),
         tf.keras.layers.Dense(128, activation='relu'),
         tf.keras.layers.Dense(10)]
    )

In [9]:
with strategy.scope():
    model = create_model()
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['sparse_categorical_accuracy'])